In [ ]:
!pip install  tiktoken cohere openai langchain chromadb google-search-results  html2text

### 구글api 발행 예시
https://littlefoxdiary.tistory.com/116

### streamlit 사용 예시
https://github.com/langchain-ai/web-explorer/blob/main/web_explorer.py

###30개까지 사용 가능
## WebResearchRetriever 

### WebResearchRetriever로 만들어서 크로마에 넣으면서 그걸로 바로 검색함

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["GOOGLE_CSE_ID"] = "GOOGLE_CSE_ID"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"


In [ ]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os

In [ ]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_wonik"
)

In [ ]:
# LLM
llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

In [ ]:

# Search
search = GoogleSearchAPIWrapper()

In [ ]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
    num_search_results=10,
    #prompt = "first check https://kprofiles.com/minji-newjeans-profile/"
)

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

In [ ]:
user_input = "랭체인이 무엇인가?"
result = qa_chain({"question": user_input})
result

In [ ]:
docs_test = vectorstore.similarity_search("랭체인이란?")

In [ ]:
docs_test

#Run with logging

In [ ]:
# Run
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
user_input = "빅뱅이론 시리즈는 몇개인가?"
docs = web_research_retriever.get_relevant_documents(user_input)

In [ ]:
docs

In [ ]:
docs = vectorstore.similarity_search("빅뱅이론 시리즈 11 주인공")

In [ ]:
docs

#Generate answer using retrieved docs

In [ ]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")

user_input = "빅뱅이론 셸던 배우 이름"

output = chain(
    {"input_documents": docs, "question": user_input}, return_only_outputs=True
)
output["output_text"]



#LLM Chain - PROMPT

In [ ]:
import os
import re
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [ ]:
class LineList(BaseModel):
    """List of questions."""

    lines: List[str] = Field(description="Questions")


class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""

    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)

In [ ]:
# LLMChain
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search
    results. Generate FIVE Google search queries that are similar to
    this question. The output should be a numbered list of questions and each
    should have a question mark at the end: {question}""",
)

llm_chain = LLMChain(
    llm=llm,
    prompt= search_prompt,
    output_parser=QuestionListOutputParser(),
)

In [ ]:
# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain,
    search=search,
)

In [ ]:
# Run
docs = web_research_retriever_llm_chain.get_relevant_documents("Financial About Korean")

In [ ]:
docs

In [ ]:
docs = vectorstore.similarity_search("주가정보")

In [ ]:
docs

In [ ]:
from langchain.chains import VectorDBQA
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=vectorstore)

In [ ]:
query = "What is the financial information for Korea?"
qa.run(query)